In [14]:
using LinearAlgebra, Random, Statistics

In [16]:
# generate random data matrix
n,d = 6,4
X = randn(n,d)

# optional: give it linearly dependent columns
X[:,3] = X[:,2]
X

6×4 Array{Float64,2}:
  0.573077   0.468126    0.468126    0.518838
  0.699222   0.0420342   0.0420342   0.12317
 -0.156645   0.374166    0.374166    0.0239568
  0.310086  -0.764711   -0.764711   -0.360399
 -0.507746   0.543277    0.543277   -2.1375
  3.18472   -0.58651    -0.58651    -1.86139

In [17]:
U,σ,V = svd(X)

SVD{Float64,Float64,Array{Float64,2}}
U factor:
6×4 Array{Float64,2}:
 -0.00539504  -0.190392   -0.642291  -0.0379604
 -0.122173    -0.169425   -0.241944  -0.701953
  0.0663229    0.101954   -0.298173  -0.462216
 -0.177398    -0.0991203   0.651604  -0.500171
 -0.152621     0.95091    -0.081925  -0.117032
 -0.962228    -0.102945   -0.093368   0.168255
singular values:
4-element Array{Float64,1}:
 3.918752252818832
 2.3622345810143344
 1.4211344525401286
 7.032906093625886e-17
Vt factor:
4×4 Array{Float64,2}:
 -0.801491   0.161851   0.161851   0.552467
 -0.459291   0.251746   0.251746  -0.813818
 -0.382967  -0.640647  -0.640647  -0.180221
  0.0       -0.707107   0.707107   5.55112e-17

In [18]:
U'*U

4×4 Array{Float64,2}:
 1.0           8.68792e-17   6.79998e-17   6.41545e-17
 8.68792e-17   1.0          -4.68924e-17  -1.55497e-16
 6.79998e-17  -4.68924e-17   1.0          -1.31907e-16
 6.41545e-17  -1.55497e-16  -1.31907e-16   1.0

In [19]:
U*U'

6×6 Array{Float64,2}:
  0.450257    0.214961    0.189291   -0.379704   -0.12316     0.0783736
  0.214961    0.594906    0.371219    0.231911   -0.0404891   0.0394824
  0.189291    0.371219    0.317344    0.0150248   0.165348   -0.124244
 -0.379704    0.231911    0.0150248   0.716054   -0.0620265   0.0359058
 -0.12316    -0.0404891   0.165348   -0.0620265   0.94793     0.0369224
  0.0783736   0.0394824  -0.124244    0.0359058   0.0369224   0.973508

In [20]:
V'*V

4×4 Array{Float64,2}:
  1.0          8.68884e-17  1.27238e-17  -2.13768e-17
  8.68884e-17  1.0          5.38631e-17   5.32179e-17
  1.27238e-17  5.38631e-17  1.0           8.5981e-17
 -2.13768e-17  5.32179e-17  8.5981e-17    1.0

In [21]:
V*V'

4×4 Array{Float64,2}:
 1.0          2.29502e-16  1.73991e-16  1.815e-16
 2.29502e-16  1.0          1.21169e-16  1.75265e-16
 1.73991e-16  1.21169e-16  1.0          7.33583e-17
 1.815e-16    1.75265e-16  7.33583e-17  1.0

In [22]:
σ

4-element Array{Float64,1}:
 3.918752252818832
 2.3622345810143344
 1.4211344525401286
 7.032906093625886e-17

In [23]:
# if we have a linearly dependent column, 
# decomposition is just as good if we ignore the 0 in sigma and reduce r by 1
println("Error of full rank svd: ", norm(X - U*diagm(σ)*V'))
for k=3:-1:1
    println("Error of rank $k approximation: ", norm(X - U[:,1:k]*diagm(σ[1:k])*(V[:,1:k])'))
end

Error of full rank svd: 3.222488743844374e-15
Error of rank 3 approximation: 3.221816314104331e-15
Error of rank 2 approximation: 1.4211344525401282
Error of rank 1 approximation: 2.7567690051827887


In [24]:
# form data from noisy linear model
w♮ = randn(d)
y = X*w♮ + .1*randn(n);

In [31]:
# solve least squares problem to estimate w
w = V*diagm(σ.^(-1))*U'*y

4-element Array{Float64,1}:
 -0.24723376947937492
  1.2296831134048805e15
 -1.2296831134048812e15
 -1.4601005663102384

In [44]:
# use rank k approximation to design matrix X
# k=4 is full rank
# when design matrix X has rank 3, k=3 gives 0 error approximation
# while k=2 results in some loss of accuracy - but not much!
k = 3
w = V[:,1:k]*diagm(σ[1:k].^(-1))*(U[:,1:k])'*y

4-element Array{Float64,1}:
 -0.24723376947937492
 -0.37987541412463555
 -0.3798754141246354
 -1.3635647571638487

In [45]:
# error in normal equations 
norm(X'*X*w - X'*y)

7.188211828540968e-15

In [46]:
w[2] += 1
w[3] -= 1
norm(X'*X*w - X'*y)

7.222425309945365e-15

In [47]:
w

4-element Array{Float64,1}:
 -0.24723376947937492
  0.6201245858753645
 -1.3798754141246354
 -1.3635647571638487

In [39]:
# how good is our estimate of w?
norm(w - w♮) / norm(w♮)

0.5997851656441274

In [40]:
# compute mean square error
mean((y - X*w).^2)

0.010609982025196208

In [41]:
# let's use the shorthand
# backslash finds least norm solution to normal eqns
# using SVD when design matrix X is rank deficient
w_backslash = X \ y
norm(w_backslash - w)

9.646809571707696e-16